In [33]:
import pandas as pd
import statsmodels.formula.api as smf
import scipy.stats as stats
from statsmodels.iolib.summary2 import summary_col

data_w=pd.read_excel("C:\\Users\\王浩晨\\Desktop\\实证文件\\panel\\数据.xlsx")
data_h=data_w[['From','last','last_log','past_experience_budget_ratio','past_experience_budget_delta','past_experience_delta','province','Fiscal_transparency_lagged','Fiscal_transparency','Fiscal_transparency_log','industry_structure','Regional_GDP_log','perGDP_log','Fiscal_autonomy','population_log','city','year','past_experience_ratio','male','age','education']]
data_h=data_h.set_index(['city','year'],drop=False)
data=pd.DataFrame(data_h)

regression1=smf.ols(formula='Fiscal_transparency_lagged~past_experience_delta+male+age+education+industry_structure+Fiscal_autonomy+population_log+perGDP_log+C(year)+C(city)',data=data)
result_reg1=regression1.fit()

data.loc[data['From'].isin([0, 1]), 'past_experience_delta'] = 0
print("past_experience = 0 的样本数：", len(data[data['past_experience_delta'] == 0]))
print("占比：{:.2%}".format(len(data[data['past_experience_delta'] == 0]) / len(data)))

regression2=smf.ols(formula='Fiscal_transparency_lagged~past_experience_delta+male+age+education+industry_structure+Fiscal_autonomy+population_log+perGDP_log+C(year)+C(city)',data=data)
result_reg2=regression2.fit()

data = data[~data['From'].isin([0, 1])].copy()

regression3=smf.ols(formula='Fiscal_transparency_lagged~past_experience_delta+male+age+education+industry_structure+Fiscal_autonomy+population_log+perGDP_log+C(year)+C(city)',data=data)
result_reg3=regression3.fit()

result=summary_col([result_reg1,result_reg2,result_reg3],
            model_names=['Model1','Model2','Model3'],                                       
            stars=True,regressor_order=['Intercept','last','last_log','past_experience_delta','past_experience_pos','past_experience_neg','male','age','education','industry_structure','Fiscal_autonomy','population_log','perGDP_log'],
            drop_omitted=True,info_dict={'':lambda x:'',
                                         '':lambda x:'',
                                         'Observations':lambda x:str(int(x.nobs)),
                                         })

print(result)

past_experience = 0 的样本数： 736
占比：31.63%

                         Model1      Model2      Model3  
---------------------------------------------------------
Intercept             91.8594***  89.7743***  145.2853***
                      (34.7588)   (34.7601)   (45.8373)  
past_experience_delta 0.0440**    0.0336*     0.0486**   
                      (0.0186)    (0.0199)    (0.0224)   
male                  0.2202      0.1765      0.8326     
                      (1.2499)    (1.2505)    (1.7169)   
age                   0.1620      0.1661*     0.1503     
                      (0.0991)    (0.0992)    (0.1289)   
education             0.9065      0.9235      0.5922     
                      (0.9910)    (0.9916)    (1.2723)   
industry_structure    0.0381      0.0422      -0.1761    
                      (0.2728)    (0.2730)    (0.3129)   
Fiscal_autonomy       7.8640      7.9646      6.6591     
                      (5.2957)    (5.2989)    (6.6822)   
population_log        -43.5502*

In [16]:
import pandas as pd
import statsmodels.formula.api as smf
import scipy.stats as stats
from statsmodels.iolib.summary2 import summary_col
import numpy as np

data_w=pd.read_excel("C:\\Users\\王浩晨\\Desktop\\实证文件\\panel\\数据.xlsx")
data_h=data_w[['From','last','last_log','past_experience_budget_ratio','past_experience_budget_delta','past_experience_delta','province','Fiscal_transparency_lagged','Fiscal_transparency','Fiscal_transparency_log','industry_structure','Regional_GDP_log','perGDP_log','Fiscal_autonomy','population_log','city','year','past_experience_ratio','male','age','education']]
data_h=data_h.set_index(['city','year'],drop=False)
data=pd.DataFrame(data_h)

data.loc[data['past_experience_delta'] >= 0, 'past_experience_delta_pos'] = data['past_experience_delta']
data.loc[data['past_experience_delta'] < 0, 'past_experience_delta_pos'] = 0

data.loc[data['past_experience_delta'] >= 0, 'past_experience_delta_neg'] = 0
data.loc[data['past_experience_delta'] < 0, 'past_experience_delta_neg'] = data['past_experience_delta']

data['past_experience_delta_pos_log'] = np.where(
    data['past_experience_delta_pos'] == 0, 
    0,  
    np.where(
        (data['past_experience_delta_pos'] > 0) & (data['past_experience_delta_pos'] <= 1), 
        np.log10(1), 
        np.log10(data['past_experience_delta_pos'])  
    )
)

data['past_experience_delta_neg_log'] = np.where(
    data['past_experience_delta_neg'] == 0, 
    0,  
    np.where(
        (-1 <= data['past_experience_delta_neg']) & (data['past_experience_delta_neg'] < 0), 
        np.log10(1),  
        np.where(
            data['past_experience_delta_neg'] < -1,
            np.log10(-data['past_experience_delta_neg']), 
            0  
        )
    )
)

data['Fiscal_transparency_lagged_log'] = np.log10(data['Fiscal_transparency_lagged'])

regression1 = smf.ols(
    formula='Fiscal_transparency_lagged_log ~ past_experience_delta_pos_log + past_experience_delta_neg_log + male + age + education + industry_structure + Fiscal_autonomy + population_log + perGDP_log + C(year) + C(city)',
    data=data
)
result_reg1=regression1.fit()

regression2 = smf.ols(
    formula='Fiscal_transparency_lagged_log ~ past_experience_delta_pos_log  + male + age + education + industry_structure + Fiscal_autonomy + population_log + perGDP_log + C(year) + C(city)',
    data=data
)
result_reg2=regression2.fit()

regression3 = smf.ols(
    formula='Fiscal_transparency_lagged_log ~ past_experience_delta_neg_log + male + age + education + industry_structure + Fiscal_autonomy + population_log + perGDP_log + C(year) + C(city)',
    data=data
)
result_reg3=regression3.fit()

result=summary_col([result_reg1,result_reg2,result_reg3],
            model_names=['Model1','Model2','Model3'],                                       
            stars=True,regressor_order=['Intercept','past_experience_delta_pos_log','past_experience_delta_neg_log','male','age','education','industry_structure','Fiscal_autonomy','population_log','perGDP_log'],
            drop_omitted=True,info_dict={'':lambda x:'',
                                         '':lambda x:'',
                                         'Observations':lambda x:str(int(x.nobs)),
                                         })

print(result)

c:\Users\王浩晨\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\王浩晨\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)



                                Model1     Model2     Model3  
--------------------------------------------------------------
Intercept                     2.2862***  2.2866***  2.2576*** 
                              (0.4511)   (0.4510)   (0.4510)  
past_experience_delta_pos_log 0.0154     0.0142*              
                              (0.0094)   (0.0073)             
past_experience_delta_neg_log 0.0021                -0.0085   
                              (0.0104)              (0.0081)  
male                          0.0073     0.0075     0.0081    
                              (0.0162)   (0.0162)   (0.0162)  
age                           0.0015     0.0015     0.0014    
                              (0.0013)   (0.0013)   (0.0013)  
education                     0.0176     0.0176     0.0175    
                              (0.0129)   (0.0129)   (0.0129)  
industry_structure            0.0003     0.0003     0.0002    
                              (0.0035)   (0.0035)   (0

In [17]:
import pandas as pd
import statsmodels.formula.api as smf
import scipy.stats as stats
from statsmodels.iolib.summary2 import summary_col
import numpy as np

data_w=pd.read_excel("C:\\Users\\王浩晨\\Desktop\\实证文件\\panel\\数据.xlsx")
data_h=data_w[['From','last','last_log','past_experience_budget_ratio','past_experience_budget_delta','past_experience_delta','province','Fiscal_transparency_lagged','Fiscal_transparency','Fiscal_transparency_log','industry_structure','Regional_GDP_log','perGDP_log','Fiscal_autonomy','population_log','city','year','past_experience_ratio','male','age','education']]
data_h=data_h.set_index(['city','year'],drop=False)
data=pd.DataFrame(data_h)

data.loc[data['From'].isin([0, 1]), 'past_experience_delta'] = 0

data.loc[data['past_experience_delta'] >= 0, 'past_experience_delta_pos'] = data['past_experience_delta']
data.loc[data['past_experience_delta'] < 0, 'past_experience_delta_pos'] = 0

data.loc[data['past_experience_delta'] >= 0, 'past_experience_delta_neg'] = 0
data.loc[data['past_experience_delta'] < 0, 'past_experience_delta_neg'] = data['past_experience_delta']

data['past_experience_delta_pos_log'] = np.where(
    data['past_experience_delta_pos'] == 0, 
    0,  
    np.where(
        (data['past_experience_delta_pos'] > 0) & (data['past_experience_delta_pos'] <= 1), 
        np.log10(1), 
        np.log10(data['past_experience_delta_pos'])  
    )
)

data['past_experience_delta_neg_log'] = np.where(
    data['past_experience_delta_neg'] == 0, 
    0,  
    np.where(
        (-1 <= data['past_experience_delta_neg']) & (data['past_experience_delta_neg'] < 0), 
        np.log10(1),  
        np.where(
            data['past_experience_delta_neg'] < -1,
            np.log10(-data['past_experience_delta_neg']), 
            0  
        )
    )
)

data['Fiscal_transparency_lagged_log'] = np.log10(data['Fiscal_transparency_lagged'])

regression1 = smf.ols(
    formula='Fiscal_transparency_lagged_log ~ past_experience_delta_pos_log + past_experience_delta_neg_log + male + age + education + industry_structure + Fiscal_autonomy + population_log + perGDP_log + C(year) + C(city)',
    data=data
)
result_reg1=regression1.fit()

regression2 = smf.ols(
    formula='Fiscal_transparency_lagged_log ~ past_experience_delta_pos_log  + male + age + education + industry_structure + Fiscal_autonomy + population_log + perGDP_log + C(year) + C(city)',
    data=data
)
result_reg2=regression2.fit()

regression3 = smf.ols(
    formula='Fiscal_transparency_lagged_log ~ past_experience_delta_neg_log + male + age + education + industry_structure + Fiscal_autonomy + population_log + perGDP_log + C(year) + C(city)',
    data=data
)
result_reg3=regression3.fit()

result=summary_col([result_reg1,result_reg2,result_reg3],
            model_names=['Model1','Model2','Model3'],                                       
            stars=True,regressor_order=['Intercept','past_experience_delta_pos_log','past_experience_delta_neg_log','male','age','education','industry_structure','Fiscal_autonomy','population_log','perGDP_log'],
            drop_omitted=True,info_dict={'':lambda x:'',
                                         '':lambda x:'',
                                         'Observations':lambda x:str(int(x.nobs)),
                                         })

print(result)

c:\Users\王浩晨\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\王浩晨\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)



                                Model1     Model2     Model3  
--------------------------------------------------------------
Intercept                     2.2700***  2.2702***  2.2456*** 
                              (0.4516)   (0.4515)   (0.4509)  
past_experience_delta_pos_log 0.0078     0.0084               
                              (0.0080)   (0.0075)             
past_experience_delta_neg_log -0.0019               -0.0051   
                              (0.0090)              (0.0084)  
male                          0.0073     0.0071     0.0077    
                              (0.0163)   (0.0162)   (0.0162)  
age                           0.0015     0.0015     0.0015    
                              (0.0013)   (0.0013)   (0.0013)  
education                     0.0178     0.0179     0.0175    
                              (0.0129)   (0.0129)   (0.0129)  
industry_structure            0.0003     0.0002     0.0003    
                              (0.0035)   (0.0035)   (0

In [32]:
import pandas as pd
import statsmodels.formula.api as smf
import scipy.stats as stats
from statsmodels.iolib.summary2 import summary_col
import numpy as np

data_w=pd.read_excel("C:\\Users\\王浩晨\\Desktop\\实证文件\\panel\\数据.xlsx")
data_h=data_w[['From','last','last_log','past_experience_budget_ratio','past_experience_budget_delta','past_experience_delta','province','Fiscal_transparency_lagged','Fiscal_transparency','Fiscal_transparency_log','industry_structure','Regional_GDP_log','perGDP_log','Fiscal_autonomy','population_log','city','year','past_experience_ratio','male','age','education']]
data=pd.DataFrame(data_h)

data = data[data['past_experience_delta'] != 0].copy()

data['treat_pos'] = (data['past_experience_delta'] > 0).astype(int)

city_has_positive = data.groupby('city')['past_experience_delta'].apply(lambda x: (x > 0).any())

data['post_pos'] = data['city'].map(city_has_positive).astype(int)

data['treat_neg'] = (data['past_experience_delta'] < 0).astype(int)

city_has_negtive = data.groupby('city')['past_experience_delta'].apply(lambda x: (x < 0).any())

data['post_neg'] = data['city'].map(city_has_negtive).astype(int)

data['Fiscal_transparency_lagged_log'] = np.log10(data['Fiscal_transparency_lagged'])

regression1 = smf.ols(
    formula='Fiscal_transparency_lagged ~ treat_pos * post_pos + male + age + education + industry_structure + Fiscal_autonomy + population_log + perGDP_log + C(year) + C(city)',
    data=data
)
result_reg1=regression1.fit()

regression2 = smf.ols(
    formula='Fiscal_transparency_lagged ~ treat_neg * post_neg + male + age + education + industry_structure + Fiscal_autonomy + population_log + perGDP_log + C(year) + C(city)',
    data=data
)
result_reg2=regression2.fit()

result=summary_col([result_reg1,result_reg2],
            model_names=['Model1','Model2','Model3'],                                       
            stars=True,regressor_order=['Intercept','treat_pos','post_pos','treat_pos:post_pos','treat_neg','post_neg','treat_neg:post_neg','male','age','education','industry_structure','Fiscal_autonomy','population_log','perGDP_log'],
            drop_omitted=True,info_dict={'':lambda x:'',
                                         '':lambda x:'',
                                         'Observations':lambda x:str(int(x.nobs)),
                                         })

print(result)


                      Model1      Model2  
------------------------------------------
Intercept          139.9831*** 120.1591***
                   (43.2783)   (40.4707)  
treat_pos          0.1468                 
                   (0.3298)               
post_pos           -48.0402***            
                   (10.8584)              
treat_pos:post_pos 0.1468                 
                   (0.3298)               
treat_neg                      -0.1468    
                               (0.3298)   
post_neg                       -27.9226***
                               (8.1373)   
treat_neg:post_neg             -0.1468    
                               (0.3298)   
male               0.1032      0.1032     
                   (1.2533)    (1.2533)   
age                0.1612      0.1612     
                   (0.1001)    (0.1001)   
education          1.1171      1.1171     
                   (0.9986)    (0.9986)   
industry_structure 0.0433      0.0433     
          